In [1]:
import urllib
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

DATASET = "VDH-COVID-19-PublicUseDataset-Cases.csv"
COUNTIES = frozenset({"Alexandria", "Arlington","Fairfax"})

data = pd.read_csv(os.path.join("./data", DATASET))
data["Datetime"] = pd.to_datetime(data["Report Date"])

data.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,Datetime
0,6/30/2020,51001,Accomack,Eastern Shore,1042,71,14,2020-06-30
1,6/30/2020,51003,Albemarle,Thomas Jefferson,401,28,8,2020-06-30
2,6/30/2020,51005,Alleghany,Alleghany,31,5,2,2020-06-30
3,6/30/2020,51007,Amelia,Piedmont,44,11,1,2020-06-30
4,6/30/2020,51009,Amherst,Central Virginia,36,3,0,2020-06-30


In [12]:
def get_daily_cases_moving_avg(data, counties, avg=7):
    total_cases = pd.DataFrame(
        data[data["Locality"].isin(counties)].groupby("Report Date")["Total Cases"].agg("sum")
    )
    total_cases.reset_index(inplace=True)
    total_cases["Datetime"] = pd.to_datetime(total_cases["Report Date"])
    total_cases.sort_values(by="Datetime", inplace=True)

    total_cases["Daily"] = total_cases["Total Cases"].diff()
    total_cases[f"{avg} day avg"] = total_cases["Daily"].rolling(avg).mean()

    total_cases["pct change"] = total_cases[f"{avg} day avg"].pct_change()
    
    return total_cases


def plot_case_counts(dataframes):
    sns.set(style="white", palette="muted", color_codes=True)
    f, axes = plt.subplots(1, 2, figsize=(20,10))
    
    for label, data in dataframes.items():
        pct_change = data["pct change"].to_list()[-1]
        left = sns.lineplot(x="Datetime", y="Total Cases", data=data, ax=axes[0],label=label)
        right = sns.lineplot(
            x="Datetime", y="7 day avg", data=data, ax=axes[1], label=f"{label} {pct_change*100:.2f}%")

    left.set_title("Cummulative Casses")
    right.set_title("7 day moving average")
    title = (f"Total Cases as of {data.iloc[-1]['Report Date']}\n"
             f"for {', '.join([county for county in dataframes.keys() if county!='Total cases'])}"
            )
    a = f.suptitle(title, fontsize=20)
    plt.show()

In [13]:
dataframes = {
    "Alexandria": get_daily_cases_moving_avg(data, {"Alexandria"}, avg=7),
    "Fairfax": get_daily_cases_moving_avg(data, {"Fairfax"}, avg=7),
    "Arlington": get_daily_cases_moving_avg(data, {"Arlington"}, avg=7),
    "Total cases": get_daily_cases_moving_avg(data, COUNTIES, avg=7)
}

In [ ]:
plot_case_counts(dataframes)